# Modify the configure_optimizers() behavior of NeuralForecast models
> Tutorial on how to achieve a full control of the `configure_optimizers()` behavior of NeuralForecast models

NeuralForecast models allow us to customize the default optimizer and learning rate scheduler behaviors via 
`optimizer`, `optimizer_kwargs`, `lr_scheduler`, `lr_scheduler_kwargs`. However this is not sufficient to support the use of [ReduceLROnPlateau](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html), for instance, as it requires the specification of `monitor` parameter.

This tutorial provides an example of how to support the use of `ReduceLROnPlateau`. 

## Load libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
from neuralforecast.utils import AirPassengersPanel

from utilsforecast.plotting import plot_series

## Data

We use the AirPassengers dataset for the demonstration of conformal prediction.


In [ ]:
AirPassengersPanel_train = AirPassengersPanel[AirPassengersPanel['ds'] < AirPassengersPanel['ds'].values[-12]].reset_index(drop=True)
AirPassengersPanel_test = AirPassengersPanel[AirPassengersPanel['ds'] >= AirPassengersPanel['ds'].values[-12]].reset_index(drop=True)
AirPassengersPanel_test['y'] = np.nan
AirPassengersPanel_test['y_[lag12]'] = np.nan

## Model training

We now train a NHITS model on the above dataset. We consider two different predictions:
1. Training using the default `configure_optimizers()`.
2. Training by overwriting the `configure_optimizers()` of the subclass of NHITS model.


In [ ]:
horizon = 12
input_size = 24

class CustomNHITS(NHITS):
    def configure_optimizers(self):
        optimizer = torch.optim.Adadelta(params=self.parameters(), rho=0.75)
        scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer=optimizer, mode='min',factor=0.5, patience=2,
        )
        scheduler_config = {
            'scheduler': scheduler,
            'interval': 'step',
            'frequency': 1,
            'monitor': 'train_loss',
            'strict': True,
            'name': None,
        }
        return {'optimizer': optimizer, 'lr_scheduler': scheduler_config}

models = [
    NHITS(h=horizon, input_size=input_size, max_steps=100, alias='NHITS-default-scheduler'),
    CustomNHITS(h=horizon, input_size=input_size, max_steps=100, alias='NHITS-ReduceLROnPlateau-scheduler'),
]
nf = NeuralForecast(models=models, freq='ME')
nf.fit(AirPassengersPanel_train)
preds = nf.predict(futr_df=AirPassengersPanel_test)

In [ ]:
plot_series(AirPassengersPanel_train, preds)

We can clearly notice the prediction outputs are different due to the change in `configure_optimizers()`.